# Maximums of Sliding Window
There’s a sliding window of size k that slides through an integer array from left to right. Create
a new array that records the largest number found in each window as it slides through.

**Example:**<br/>
Input: nums = [3, 2, 4, 1, 2, 1, 1], k = 4<br/>
Output: [4, 4, 4, 2]

# Intuition

A brute-force approach to solving this problem involves iterating through each element within a window to find the maximum value. Repeating this for each window results in an O(n * k) time complexity, as we traverse `k` elements for up to `n` windows.

The problem with this approach is that adjacent windows share many of the same elements, causing us to re-examine values that have already been considered in previous windows.

A more efficient solution can avoid this by keeping track of values within each window so that when we move to the next window, we don't need to recheck the same values. We maintain a record of only those values that have the potential to be the maximum in a future window. These are called "candidates", and any value that isn't a candidate can no longer contribute to a future maximum.

**Example**: Given the array `[3, 2, 4, 1, 2, 1, 1]` with a window size `k = 4`:

- **3**: Initially, 3 is a candidate for the current window, but once the window slides, we can ignore it since it will no longer be part of any future windows.
- **2**: 2 can never be the maximum in future windows because of the larger 4 to its right. Any window containing 2 will also contain 4, making 2 irrelevant for future maximums.
- **4**: This is the maximum value in the current window and will be part of future windows. Therefore, 4 is a candidate for future windows.
- **1**: 1 could potentially be the maximum in future windows. Even though 4 is larger in the current window, as the window slides, 1 will remain while 4 gets excluded, making 1 a valid candidate.

### Strategy

1. **Remove smaller or equal candidates**: If the new value is larger than existing candidates, discard any candidate smaller or equal to it, as they cannot be a maximum in any future window.
2. **Add the new candidate**: Once smaller candidates are removed, add the new value to the list of candidates.
3. **Remove outdated candidates**: As the window slides, discard any candidates whose index falls outside the current window.

An important observation is that the candidates are always in decreasing order. This is because each new candidate eliminates smaller and equal values to its left, ensuring that the list remains in a decreasing order. The maximum value for any window will always be the first element in the candidate list.

Thus, we need a data structure that maintains this decreasing order of values.

---

### Deque

A stack can maintain a decreasing order of values but has a limitation: it only allows access to the most recent element, making it difficult to remove outdated candidates from the beginning of the list. A **deque** (double-ended queue) is a better fit because it allows efficient addition and removal from both ends in O(1) time.

While a deque behaves like a double-ended stack, it provides the flexibility to remove outdated candidates efficiently. The deque will store tuples containing both the value and its index. The index helps track if a value has moved out of the window.

---

### Example

Given the array `[3, 2, 4, 1, 2, 1, 1]`, and an empty deque `dp = []`:

- **Start expanding the window** until it reaches size `k`. For each candidate, maintain the decreasing order in the deque before adding it.
    - Add `3`: Remove all values in the deque less than or equal to 3, then push `(3, index)` into the deque.
    - Add `2`: Remove values in the deque less than or equal to 2, then push `(2, index)` into the deque.
    - Add `4`: Remove values in the deque less than or equal to 4, then push `(4, index)` into the deque.
  
- Once the window size is `k`, we begin recording the maximum value for each window. The maximum value is always the first value in the deque, which is `4` in this case.

- **Sliding the window**: As the window slides, remove candidates from the deque whose index is before the left pointer of the window (i.e., values that are no longer within the window).

- Continue sliding the window until the end of the input array.

In [1]:
from typing import List
from collections import deque

def maximums_of_sliding_window(nums: List[int], k: int) -> List[int]:
    res = []
    dq = deque()
    left = right = 0

    while right < len(nums):
        while dq and dq[-1][0] <= nums[right]:
            dq.pop()

        dq.append((nums[right], right))

        if right - left + 1 == k:
            if dq and dq[0][1] < left:
                dq.popleft()

            res.append(dq[0][0])
            left += 1

        right += 1

    return res

### Complexity Analysis

The time complexity is O(n) because we slide over the array in linear time, and we push and pop values of nums into the deque at most once for each number, with each stack operation taking O(1) time.

The space complexity is O(k) because the deque can store up to k elements. Note, we don't consider res in the space complexity.